In [1]:
from metaworld import MT1
import numpy as np
from PIL import Image
import cv2
from metaworld.policies.sawyer_shelf_place_v2_policy import SawyerShelfPlaceV2Policy as p
from numcodecs import Zstd
import time
import zarr

from pyvirtualdisplay import Display

def resize(img):
    return cv2.resize(img,(224,224),interpolation=cv2.INTER_AREA)

def batch_resize(batch_images):
  N=batch_images.shape[0]
  processed_images=np.zeros((N,224,224,3),dtype=np.uint8)
  for i in range(N):
    processed_images[i]=resize(batch_images[i])
  return processed_images

display = Display(visible=0,backend="xvfb")
display.start()

render_mode='rgb_array'
#camera_name='corner'
camera_name='behindGripper'
#camera_name='gripperPOV'

root=zarr.open_group('./shelf_place_with_video.zarr',mode='w')
episode_ends=root.create_dataset('meta/episode_ends',shape=(0,1),chunks=(100,-1),dtype='int32')
img=root.create_dataset('data/img',shape=(0,224,224,3),chunks=(50,-1,-1,-1),dtype='uint8',compressor=Zstd(level=1))
action=root.create_dataset('data/action',shape=(0,4),chunks=(100,-1),dtype='float32')
state=root.create_dataset('data/state',shape=(0,39,),chunks=(100,-1),dtype='float32')

policy = p()
#imgs=[]
episode_end=0
for seed in range(100,105):
    mt1 = MT1('shelf-place-v2', seed)
    env = mt1.train_classes['shelf-place-v2'](render_mode=render_mode,camera_name=camera_name)
    print(len(mt1.train_tasks))
    print(len(mt1.test_tasks))
    for i in range(50):
        
        #print("train_task:",str(i))
        steps_cnt=0
        env.set_task(mt1.train_tasks[i])
        obs, info = env.reset()
        state_list=[]
        img_list=[]
        action_list=[]
        state_list.append(obs)
        img_list.append(env.render())
    
        done = False
        
        while not done:
            a = policy.get_action(obs)
            action_list.append(a)
            episode_end+=1
            steps_cnt+=1
            obs, reward, terminated, truncated, info = env.step(a)
            
            if int(info['success']) == 1:
                print("train_task:",str(i),"success")
                episode_ends.append(np.array([[episode_end]]))
                # submit
                #print("shape of state",state.shape)
                #print("shape of state list array:",np.array(state_list).shape)
                state.append(np.array(state_list),axis=0)
                img.append(batch_resize(np.array(img_list)),axis=0)
                action.append(np.array(action_list),axis=0)
                done = True
            elif terminated==True or truncated==True :
                episode_end-=steps_cnt
                done = True
            else:
                state_list.append(obs)
                img_list.append(env.render())
                #imgs.append(env.render())
    
    
print("ep_ends_shape:",episode_ends.shape)
print("state_shape:",state.shape)
print("action_shape:",action.shape)
print("img_shape:",img.shape)

'''
from skvideo.io import vwrite
from IPython.display import Video
vwrite("vis.mp4",imgs)
Video("vis.mp4",embed=True)
'''
display.stop()

50
0


/public1_data/hjl/Metaworld/metaworld/policies/policy.py:49: UserWarning: Constant(s) may be too high. Environments clip response to [-1, 1]
  warnings.warn(


train_task: 0 success
train_task: 1 success
train_task: 2 success
train_task: 3 success
train_task: 4 success
train_task: 5 success
train_task: 6 success
train_task: 7 success
train_task: 8 success
train_task: 9 success
train_task: 10 success
train_task: 11 success
train_task: 12 success
train_task: 13 success
train_task: 14 success
train_task: 15 success
train_task: 16 success
train_task: 17 success
train_task: 18 success
train_task: 19 success
train_task: 20 success
train_task: 21 success
train_task: 22 success
train_task: 23 success
train_task: 24 success
train_task: 25 success
train_task: 26 success
train_task: 27 success
train_task: 28 success
train_task: 29 success
train_task: 30 success
train_task: 31 success
train_task: 32 success
train_task: 33 success
train_task: 34 success
train_task: 35 success
train_task: 36 success
train_task: 37 success
train_task: 38 success
train_task: 39 success
train_task: 40 success
train_task: 41 success
train_task: 42 success
train_task: 43 succes

In [3]:
from skvideo.io import vwrite
from IPython.display import Video
import zarr
import os

print(os.getcwd())

root=zarr.open_group('./shelf_place_with_video.zarr',mode='r')
img=root['data']['img']
print(img.shape)

vwrite("vis.mp4",img[400:1000])
Video("vis.mp4",embed=True)
print('done')


/public1_data/hjl/workshop/experiment
(21595, 224, 224, 3)
done
